In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim



In [3]:

x_train = np.random.rand(1000, 3)
y_train = np.random.rand(1000, 1)

In [19]:
x_train = torch.as_tensor(x_train,  dtype = torch.float32)
y_train = torch.as_tensor(y_train, dtype = torch.float32)

x_train.requires_grad = True
y_train.requires_grad = True

In [130]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(3, 8)
        self.fc2 = nn.Linear(8, 8)
        self.fc3 = nn.Linear(8, 3)
        

    def residual(self, x):
        r = self.fc3(F.tanh(self.fc2(F.tanh(self.fc1(x)))))
        return r

    def forward(self, x):
        if x.dim == 1: return self.residual(x)
        else:
            r = self.residual(x)
            return torch.stack([torch.dot(r.T, r) for r in r]).squeeze()


net = Net()
net = net.float()

In [135]:
b = net(x_train)


/home/amit/.local/lib/python3.6/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [136]:
b.shape

torch.Size([1000])

In [143]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

criterion = nn.MSELoss()


In [144]:
epochs = 5

In [145]:
for epoch in range(epochs):
    total_loss = 0
    for inputs, target in zip(x_train, y_train):   
        optimizer.zero_grad()
        y_hat = net(x_train.float())
        loss = ((y_hat-target)**2).mean()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print("Epoch : ", epoch , "Loss : ", total_loss )
       
        
    

/home/amit/.local/lib/python3.6/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch :  0 Loss :  84.73806494160635
Epoch :  1 Loss :  84.64371074701012
Epoch :  2 Loss :  84.58579232975717
Epoch :  3 Loss :  84.53345773228466
Epoch :  4 Loss :  84.48542998566882


Jacobians and Hessians


https://gist.github.com/sbarratt/37356c46ad1350d4c30aefbd488a4faa

In [13]:
def jacobian(y, x, create_graph=False):                                                               
    jac = []                                                                                          
    flat_y = y.reshape(-1)                                                                            
    grad_y = torch.zeros_like(flat_y)                                                                 
    for i in range(len(flat_y)):                                                                      
        grad_y[i] = 1.                                                                                
        grad_x, = torch.autograd.grad(flat_y, x, grad_y, retain_graph=True, create_graph=create_graph)
        jac.append(grad_x.reshape(x.shape))                                                           
        grad_y[i] = 0.                                                                                
    return torch.stack(jac).reshape(y.shape + x.shape)                                                
                                                                                                      
def hessian(y, x):                                                                                    
    return jacobian(jacobian(y, x, create_graph=True), x)

In [17]:
x = torch.ones(1, 3, device = device)
x.requires_grad=True

In [18]:
jacobian(net(x), x)

tensor([[[[-0.3738,  1.0299, -0.2214]]]], device='cuda:0')